# Transform and Forward Messages

This notebook consumes messages from one Kafka topic, modifies them, and sends them to a new topic in Confluent Cloud.

Make sure your `.env` file is set up correctly with:
- `BOOTSTRAP_SERVERS`
- `SASL_USERNAME`
- `SASL_PASSWORD`
- `TOPIC_NAME` (source)
- `FORWARD_TOPIC` (destination)

In [ ]:
import os
import json
from kafka import KafkaConsumer, KafkaProducer
from dotenv import load_dotenv

load_dotenv()

source_topic = os.getenv("TOPIC_NAME")
dest_topic = os.getenv("FORWARD_TOPIC")

## Step 1: Setup Kafka Consumer and Producer

In [ ]:
# Consumer setup
consumer = KafkaConsumer(
    source_topic,
    bootstrap_servers=os.getenv("BOOTSTRAP_SERVERS"),
    security_protocol="SASL_SSL",
    sasl_mechanism="PLAIN",
    sasl_plain_username=os.getenv("SASL_USERNAME"),
    sasl_plain_password=os.getenv("SASL_PASSWORD"),
    auto_offset_reset="earliest",
    enable_auto_commit=True,
    value_deserializer=lambda m: m.decode('utf-8'),
    consumer_timeout_ms=5000
)

# Producer setup
producer = KafkaProducer(
    bootstrap_servers=os.getenv("BOOTSTRAP_SERVERS"),
    security_protocol="SASL_SSL",
    sasl_mechanism="PLAIN",
    sasl_plain_username=os.getenv("SASL_USERNAME"),
    sasl_plain_password=os.getenv("SASL_PASSWORD"),
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

## Step 2: Consume, Transform, and Produce

In [ ]:
for msg in consumer:
    print("Original:", msg.value)
    transformed = {
        "original": msg.value,
        "edited": msg.value.upper(),
        "length": len(msg.value)
    }
    producer.send(dest_topic, transformed)

producer.flush()
print(f"Messages have been forwarded to topic: {dest_topic}")